In [30]:
import urllib
import requests
from pandas.io.json import json_normalize
import json
import pandas as pd
import datetime
import time
import math
import numpy as np

In [2]:
url = 'https://rsbuddy.com/exchange/summary.json'
urljson = urllib.request.urlopen(url)
jsondata = json.loads(urljson.read().decode())

In [4]:
#Get a list of all items
df = pd.read_json(path_or_buf='https://rsbuddy.com/exchange/summary.json',orient='index', convert_axes=True)
df = df[['id','name','buy_average','buy_quantity','sell_average','sell_quantity','overall_average','overall_quantity']]
data = df.sort_values(by=['id']).reset_index()
data = data.drop(labels='index',axis=1)

In [5]:
#Run this to populate a csv: itemKey
itemKey = data[['id', 'name']]
itemKey.to_csv(path_or_buf='./itemKey.csv', columns=('id','name'), index=False)

In [7]:
#Create test set of specified range number of items
data = pd.read_csv('./itemKey.csv', skiprows=[])
startNum = 0
numItems = 3
inputdata = data[startNum:startNum+numItems]
inputdata

,id,name
3,10,Cannon barrels
4,12,Cannon furnace
5,28,Insect repellent


In [10]:
#Use this to pull a specific items by name
names = ['Armadyl godsword','Fire rune', 'Soft clay']

#read the itemKey csv and parse the key for associated item ids
data = pd.read_csv('./itemKey.csv', skiprows=[])
inputdata = pd.DataFrame()
for name in names:
    inputdata = inputdata.append(data.loc[data['name'] == name])
inputdata = inputdata.reset_index().drop(labels='index',axis=1)

print("Query List:")
print(inputdata)

Query List:
      id              name
0  11802  Armadyl godsword
1    554         Fire rune
2   1761         Soft clay


In [12]:
i = 0
firtIteration = True
urlquery = pd.DataFrame(columns=['id', 'name', 'data'])
#Iterate through all items, grab data from api and append to urlquery dataframe
while(i<inputdata['id'].count()):
    key = inputdata.iloc[i]['id']
    name = inputdata.iloc[i]['name']
    if (firtIteration == True):
        print("Item Id - " + str(key) + ": " + name)
    #Attempt to get data from API, retry if HTTP error
    try:
        url = f'https://api.rsbuddy.com/grandExchange?a=graph&g=240&start=1474615279000&i={key}'
        tempDataFrame = pd.DataFrame()
        tempDataFrame = pd.read_json(path_or_buf=url,orient='records', convert_axes=False)
        urlquery = urlquery.append({'id':key, 'name': name, 'data':tempDataFrame}, ignore_index=True)
        i+=1
        #check if there are more items to run. If no more items, return message indicating the process is done
        if (i == inputdata['id'].count()):
            print("It worked! That was the last item!")
        else:
            print("It worked! On to the next one.")
        firtIteration = True
    except:
        print("Got fucked. Trying again. :)")
        time.sleep(1)
        firtIteration = False
#Reformat urlquery columns, convert timestamp from miliseconds to seconds, convert timestamp to date, and sort by date
for index, row in urlquery.iterrows():
    row['data'] = row['data'][['ts','buyingPrice','buyingCompleted','sellingPrice','sellingCompleted','overallPrice','overallCompleted']]
    row['data'] = row['data'].sort_values(by=['ts'],ascending=1).reset_index()
    count = 0 
    for ind, r in row['data'].iterrows():
        r['ts'] = r['ts']/1000
        row['data'].loc[ind, 'ts'] = int(r['ts'])
        row['data'].loc[ind, 'date'] = datetime.datetime.fromtimestamp(r['ts']).isoformat()
    row['data'] = row['data'].drop(labels='index',axis=1)

Item Id - 11802: Armadyl godsword
Got fucked. Trying again. :)
Got fucked. Trying again. :)
It worked! On to the next one.
Item Id - 554: Fire rune
Got fucked. Trying again. :)
Got fucked. Trying again. :)
Got fucked. Trying again. :)
It worked! On to the next one.
Item Id - 1761: Soft clay
Got fucked. Trying again. :)
It worked! That was the last item!


In [13]:
test_set_size = 30 #In Days - define the number of epochs to include in the test data

#create test_item object to log training and test sets
test_item = pd.DataFrame(columns=['id', 'name', 'train_x', 'train_y', 'test_x', 'test_y', 'pred_y'])
h = 0
#iterate through the urlquery results to generate training sets
while(h<urlquery['id'].count()):  
    item_id = urlquery.iloc[h]['id'] #Define the item to be queried for
    print("Item ID: " + str(item_id))
    #takes the item_id and generates the test data for the specified parameters as an array
    test_set = urlquery.loc[urlquery['id'] == item_id]
    train_x_headers = list(urlquery.iloc[0]['data'].columns.values[0:5])
    
    #for column key, uncomment this line below:
    #print(train_x_headers)

    #create training datasets
    train_x = test_set.iloc[0]['data'].iloc[0:math.ceil(test_set_size*6)].values[:,0:5]
    train_y = test_set.iloc[0]['data'].iloc[0:math.ceil(test_set_size*6)].values[:,5]
    print ("The shape of the " + test_set.iloc[0]['name'] +  " input array is: "+ str(train_x.shape))

    #create test set to estimate next 6 prices (next day of prices)
    test_x = test_set.iloc[0]['data'].iloc[math.ceil(test_set_size*6):math.ceil(test_set_size*6)+6].values[:,0:5]

    #test key to compare to Y-hat
    test_y = test_set.iloc[0]['data'].iloc[math.ceil(test_set_size*6):math.ceil(test_set_size*6)+6].values[:,5]
    test_item = test_item.append({'id':item_id, 'name': test_set.iloc[0]['name'], 'train_x':train_x, 'train_y':train_y, 'test_x':test_x, 'test_y': test_y}, ignore_index=True)
    print("Record completed at test_item position [" + str(h) + "]")
    h += 1
    #TODO: Get this all in a for loop to iterate over multiple entries

Item ID: 11802
The shape of the Armadyl godsword input array is: (180, 5)
Record completed at test_item position [0]
Item ID: 554
The shape of the Fire rune input array is: (180, 5)
Record completed at test_item position [1]
Item ID: 1761
The shape of the Soft clay input array is: (180, 5)
Record completed at test_item position [2]


In [75]:
# independent variables: Buy price, buy quantity, sell price, sell quantity, time (in seconds/year)
# dependent variable: Overall price (in percent 1-:1)
class Node:
    def __init__(self, incomingEdges=np.array([]), outgoingEdges=np.array([]), inputNodeValue=None, activation=None, bias=0):
        self.incomingEdges = incomingEdges
        self.outgoingEdges = outgoingEdges
        self.inputNodeValue = inputNodeValue
        self.activation = activation
        self.bias = bias
    
    def calculateActivation(self):
        #Iterate through all incoming edges and sum weights
        self.activation = 0
        for edge in self.incomingEdges:
            self.activation += edge.getWeightActivationProduct()
        #Add bias
        self.activation += self.bias
        #Apply activation function
        #self.activation = np.tanh(self.activation)
        
    #Getters
    def getActivation(self):
        return self.activation
    
    def getInputNodeValue(self):
        return self.inputNodeValue
    
    #Utility functions
    def appendOutgoingEdge(self, edge):
        self.outgoingEdges = np.append(self.outgoingEdges, edge)

    def appendIncomingEdge(self, edge):
        self.incomingEdges = np.append(self.incomingEdges, edge)

    
            

class Edge:
    weight = 1
    prevNode = None
    destNode = None
    
    def __init__(self, weight=1, prevNode=None, destNode=None):
        self.weight = weight
        self.prevNode = prevNode
        self.destNode = destNode
    
    def getWeight(self):
        return self.weight
    
    def getWeightActivationProduct(self):
        #If not input node, calc activation. Otherwise, return input value
        if(self.prevNode.getInputNodeValue() == None):
            
            return self.weight * self.prevNode.getActivation()
        return self.weight * self.prevNode.getInputNodeValue()
    
class NeuralNetwork:
    def __init__(self, inputNodes=np.array([]), hiddenNodes=np.array([]), outputNode=Node(), numInputNodes=5, numHiddenLayers=1, numHiddenLayerNodes=16):
        self.inputNodes = inputNodes
        self.hiddenNodes = hiddenNodes
        self.outputNode = outputNode
        self.numInputNodes = numInputNodes
        self.numHiddenLayers = numHiddenLayers
        self.numHiddenLayerNodes = numHiddenLayerNodes
        
    def generateInitialNetwork(self, inputNodeValues):
        #Generate input nodes
        for value in inputNodeValues:
            #Create new input nodes
            newNode = Node(inputNodeValue=value)
            self.inputNodes = np.append(self.inputNodes, newNode)
        #Generate hidden nodes
        for x in range(0, self.numHiddenLayerNodes):
            newNode = Node()
            self.hiddenNodes = np.append(self.hiddenNodes, newNode)
        #Generate output node
        self.outputNode = Node()
        
        #Iterate through each input node, and attach edge to each hidden node
        for inputNode in self.inputNodes:
            for hiddenNode in self.hiddenNodes:
                #Create outgoing edge with random weight
                newEdge = Edge(weight=1, prevNode=inputNode, destNode=hiddenNode)
                inputNode.appendOutgoingEdge(newEdge)
                hiddenNode.appendIncomingEdge(newEdge)
                
        #Connect each hidden layer node to the output node
        for hiddenNode in self.hiddenNodes:
            edgeToOutput = Edge(weight=1, prevNode=hiddenNode, destNode=self.outputNode)
            hiddenNode.appendOutgoingEdge(edgeToOutput)
            self.outputNode.appendIncomingEdge(edgeToOutput)
            
    def calculateNodeActivations(self):
        #Start with input nodes
        for inputNode in self.inputNodes:
            inputNode.calculateActivation()
        #Get activation of hidden layer nodes
        for hiddenNode in self.hiddenNodes:
            hiddenNode.calculateActivation()
        #Lastly, calculate output node activation
        self.outputNode.calculateActivation()
 
            
nn = NeuralNetwork()
nn.generateInitialNetwork([1,1])
nn.calculateNodeActivations()
print(nn.outputNode.getActivation())



32
